# **FAKE NEWS DETECTION**

In [1]:
import pandas as pd

In [2]:
true_df = pd.read_csv("/content/drive/MyDrive/Dataset/Project Dataset/3_Fake_News_Classifier/true.csv")

In [3]:
fake_df = pd.read_csv("/content/drive/MyDrive/Dataset/Project Dataset/3_Fake_News_Classifier/fake.csv")

In [4]:
# fake_df.isna().sum()

In [5]:
true_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [6]:
true_df["title"][3]

'FBI Russia probe helped by Australian diplomat tip-off: NYT'

In [7]:
fake_df["title"][3]

' Trump Is So Obsessed He Even Has Obama’s Name Coded Into His Website (IMAGES)'

In [8]:
max_words = true_df['title'].apply(lambda x: len(str(x).split())).max()
print("Maximum sentence length (words):", max_words)

Maximum sentence length (words): 20


In [9]:
fake_df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [10]:
max_words = fake_df['title'].apply(lambda x: len(str(x).split())).max()
print("Maximum sentence length (words):", max_words)

Maximum sentence length (words): 42


In [11]:
true_df["labels"]=1

In [12]:
fake_df["labels"]=0

In [13]:
df=pd.concat([true_df,fake_df]).reset_index(drop=True)

In [14]:
df.head()

,title,text,subject,date,labels
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [15]:
x=df.drop('labels',axis=1)
y=df["labels"]

In [16]:
#train and test split
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size=0.2, random_state=42)

In [17]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(35918, 4)
(8980, 4)
(35918,)
(8980,)


In [18]:
import tensorflow

In [19]:
from tensorflow.keras.layers import Embedding,Dense,LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [20]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [21]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [22]:
messages = x_train.copy()
m=x_test.copy()

In [23]:
messages.head()

,title,text,subject,date
36335,HILLARY LIES: Remember When Hillary Disclosed ...,This story is from 2006 and is the first in a ...,politics,"Nov 18, 2015"
12384,EU leaders likely to give go-ahead to new phas...,PARIS (Reuters) - The leaders of the European ...,worldnews,"December 13, 2017"
24419,Mark Hamill Is Back Mocking Trump By Reading ...,Star Wars icon Mark Hamill previously mocked D...,News,"January 15, 2017"
24740,Trump Gets Hilariously OWNED For Whining Abou...,The whiner-in-chief struck again with yet anot...,News,"December 21, 2016"
27039,Obama’s Labor Secretary Zaps Trump’s Foreign ...,Donald Trump doesn t have much foreign policy ...,News,"July 3, 2016"


In [24]:
messages.reset_index(inplace=True)
m.reset_index(inplace=True)

In [25]:
messages.columns

Index(['index', 'title', 'text', 'subject', 'date'], dtype='object')

In [26]:
messages.head()

,index,title,text,subject,date
0,36335,HILLARY LIES: Remember When Hillary Disclosed ...,This story is from 2006 and is the first in a ...,politics,"Nov 18, 2015"
1,12384,EU leaders likely to give go-ahead to new phas...,PARIS (Reuters) - The leaders of the European ...,worldnews,"December 13, 2017"
2,24419,Mark Hamill Is Back Mocking Trump By Reading ...,Star Wars icon Mark Hamill previously mocked D...,News,"January 15, 2017"
3,24740,Trump Gets Hilariously OWNED For Whining Abou...,The whiner-in-chief struck again with yet anot...,News,"December 21, 2016"
4,27039,Obama’s Labor Secretary Zaps Trump’s Foreign ...,Donald Trump doesn t have much foreign policy ...,News,"July 3, 2016"


In [27]:
ps = PorterStemmer()
stop_words = set(stopwords.words('english'))

def preprocess_text(parameter):
    corpus = []
    for i in range(len(parameter)):
        review = re.sub('[^a-zA-Z]', ' ', parameter["title"][i])   # remove non-letters
        review = review.lower()                       # lowercase
        review = review.split()                       # tokenize
        review = [ps.stem(word) for word in review if word not in stop_words]
        review = ' '.join(review)                     # back to sentence
        corpus.append(review)
    return corpus

In [28]:
# Preprocess train and test texts separately
train_corpus = preprocess_text(messages)
test_corpus  = preprocess_text(m)

In [29]:
# One-hot encoding (same vocab size as training)
vocab_size = 5000
train_onehot = [one_hot(words, vocab_size) for words in train_corpus]
test_onehot  = [one_hot(words, vocab_size) for words in test_corpus]

In [30]:
# Padding (same sent_length)
sent_length = 50
x_train = pad_sequences(train_onehot, padding="pre", maxlen=sent_length)
x_test  = pad_sequences(test_onehot, padding="pre", maxlen=sent_length)

In [31]:
# ### Dataset Preprocessing
# from nltk.stem.porter import PorterStemmer
# ps = PorterStemmer()
# corpus = []
# for i in range(0, len(messages)):
#     review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
#     review = review.lower()
#     review = review.split()

#     review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
#     review = ' '.join(review)
#     corpus.append(review)

# **One Hot Representation**

In [32]:
# vocab_size=5000
# onehot_repr=[one_hot(words,vocab_size)for words in corpus]
# onehot_repr

In [33]:
# sent_length=50
# padding_sequences = pad_sequences(onehot_repr,padding="pre",maxlen=sent_length)

# **Model Formation**

In [34]:
#LSTM layers #Model Formation
models = Sequential()
models.add(Embedding(vocab_size,40))
models.add(LSTM(100))
models.add(Dense(1,activation="sigmoid"))
models.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [35]:
print(x_train.shape, x_train.dtype)
print(y_train.shape, y_train.dtype)
print(x_test.shape, x_test.dtype)
print(y_test.shape, y_test.dtype)

(35918, 50) int32
(35918,) int64
(8980, 50) int32
(8980,) int64


In [37]:
# Model fit
# models.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=30,batch_size=64)

from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor="val_loss",    # watch validation loss
    patience=3,            # stop if no improvement for 3 epochs
    restore_best_weights=True
)

history = models.fit(
    x_train,y_train,
    validation_data=[x_test,y_test],
    epochs=50,             # set a high max
    callbacks=[early_stop]
)


Epoch 1/50
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.9967 - loss: 0.0095 - val_accuracy: 0.9383 - val_loss: 0.3922
Epoch 2/50
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.9971 - loss: 0.0083 - val_accuracy: 0.9400 - val_loss: 0.3917
Epoch 3/50
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.9980 - loss: 0.0053 - val_accuracy: 0.9315 - val_loss: 0.3866
Epoch 4/50
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.9980 - loss: 0.0066 - val_accuracy: 0.9383 - val_loss: 0.3722
Epoch 5/50
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.9989 - loss: 0.0046 - val_accuracy: 0.9405 - val_loss: 0.3935
Epoch 6/50
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.9996 - loss: 0.0014 - val_accuracy: 0.9366 - val_loss: 0.4165
Epoch 7/50
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.9988 - loss: 0.0037 - val_accuracy: 0.9424 - val_loss: 0.4030


In [38]:
models.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 50, 40)         │       200,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 100)            │        56,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 769,505 (2.94 MB)

 Trainable params: 256,501 (1001.96 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 513,004 (1.96 MB)

# **Adding Dropout**

In [39]:
# from tensorflow.keras.layers import Dropout
# ## Creating model
# embedding_vector_features=40
# model=Sequential()
# model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
# model.add(Dropout(0.3))
# model.add(LSTM(100))
# model.add(Dropout(0.3))
# model.add(Dense(1,activation='sigmoid'))
# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# ***Performance Metrics and Accuracy***

In [40]:
# model test
y_pred = (models.predict(x_test)>0.5).astype("int32")

281/281 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [41]:
#Confusion Matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[4441,  209],
       [ 345, 3985]])

In [42]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9383073496659243

In [43]:

# Save as .keras file
models.save("fake_news.keras")

In [44]:
# from google.colab import files

# # Download .h5 file
# files.download("fake_news.h5")